In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.metrics import confusion_matrix

In [2]:
def splitDataset(dataset, trainRatio):
    trainSize = int(len(dataset) * trainRatio)
    trainSet = []
    copy = list(dataset.iloc[:,:].values)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [np.array(trainSet), np.array(copy)]

def calc_centroids(dataset, labels):
    dataset=pd.DataFrame(dataset)
    centroids=[]
    for label in labels:
        centroids.append((list((dataset[dataset['Target']==label].iloc[:,:-1]).sum()/len(dataset[dataset['Target']==label])),label))
    return(centroids)

def calc_products(dataset,amostra,labels):
    products=[]
    for label in labels:
        cov_mat=dataset[dataset.Target==label].iloc[:,:-1].cov()
        inv_cov_mat=np.linalg.inv(cov_mat)
        centroid = calc_centroids(dataset,[label])
        x=amostra-centroid[0][0]
        product=np.dot(np.dot(x,inv_cov_mat),x.transpose())
        products.append((product, centroid[0][1]))
    return(products)        

def att_class(dataset,amostra,labels):
    dataset=pd.DataFrame(dataset)    
    products=calc_products(dataset,amostra,labels)
    products.sort()
    return(products[0][1])

def make_pred(dataset,amostras):
    labels = list(dataset.iloc[:,-1].unique())
    preds=[]
    for amostra in amostras:
        preds.append(att_class(dataset, amostra,labels))
    return(preds)

In [3]:
def prepare_to_remove(dataset):
    """
    Calcula algumas estatísticas para a remoção de outliers do dataset.
    
    Input
    ----------
    dataset: 
    Dataset a ser preparado para remoção de outliers.
    
    Output
    ----------
    cols:
    Lista contendo os nomes das colunas dos datasets.
  
    means:
    Lista contendo as médias das colunas dos datasets.
  
    stds:
    Lista contendo os desios padrões das colunas dos datasets.
  
    """
    cols, means, stds = [],[],[]
    for col in dataset.columns:
        try:
            cols.append(col)
            means.append(dataset[col].mean())
            stds.append(dataset[col].std())
        except TypeError:
            means.append(np.nan)
            stds.append(np.nan)
            print(f'Coluna {col} possui valores em formato não númerico!')
    return(cols, means, stds)

def remove_outliers(dataset):
    """
    Soma todos os elementos de um vetor, exceto o que está na posição n.
    
    
    Input
    ----------
    dataset: 
    Dataset para remoção de outliers.
    
    Output
    ----------
    dataset:
    O dataset da entrada, agora com os outliers removidos
    
    
    Obs.: São consideradas outliers da variável x, observações além do intervalo: mean(x) ± 2*std(x).
    """
    cols, means, stds = prepare_to_remove(dataset)
    k=0
    for col in cols:
        dataset = dataset.loc[(dataset[col] > means[k]-2*stds[k]) & (dataset[col] < means[k]+2*stds[k])]
        k=k+1
    return(dataset)

# Pré-Processamento dos Dados
<br>

## Remoção de Outliers
Antes da implementação do Classificador Quadrático em si, fazemos um tratamento nos dados para remover informações melhor o desempenho do classificador.<br>
    
Primeiramente carregamos o dataset diretamente do github e removemos seus outliers.
Foram considerados outliers da característica x qualquer valor além do intervalo $\bar x \pm 2s_x$.<br>

Onde $\bar x$ é a média amostral de x e $s_x$ é o desvio padrão amostral de x.<br>

## Importância das Características
<p>
Após a remoção dos outliers, separamos o conjunto em conjuntos de treino e teste e utilizamos o algoritmo classificador Random Forest nesses conjuntos para determinar a importância  de cada variável.<br>
    
Para um melhor escolha e menor enviesamento, foi feito um loop para calcular a importância de cada característica 50 vezes, calculando-se a partir daí a importância média de cada característica.<br>

Com a importância média de cada característica, foi calculada a média desses valores e foram eliminados do modelo aquelas características que apresentavam importância abaixo da importância média.<br>

Esta medida foi utilizada também para diminuir o custo computacional, tornando o algoritmo mais rápido.
</p>
<b> Tabela com as importâncias </b>

## Análise de Correlação
Em seguida, foi feita uma análise de correlação entre as variáveis restantes do modelo, para ver quão correlacionadas elas são.

<b> Imagem com o heatmap de correlação </b>

Como visto na imagem $\ref{fig:heatmap_corr}$, todas as características MA_Detection_alpha, que representam a presença de anomalias de massa em diferentes degraus de liberdade, estão extremamente correlacionadas, então a MA_Detection_alpha-0.5 é mantida por possuir a maior importância pro modelo e as outras são removidas, pois não há necessidade de 5 características fornecendo as mesmas informações. <br>

Novamente, a remoção de mais características ajuda na performance do modelo.


In [4]:
path='https://raw.githubusercontent.com/rhanielmx/RecPad/master/messidor_features.csv'
trainRatio=0.5

data=pd.read_csv(path)
data=remove_outliers(data)
feature_importances=[]
cols_names=[]
for i in range(50):
    train_set, test_set = splitDataset(dataset=data,trainRatio=trainRatio)
    X_train,y_train=[train_set[i][:-1] for i in range(train_set.shape[0])],[train_set[i][-1] for i in range(train_set.shape[0])]
    X_test,y_test=[test_set[i][:-1] for i in range(test_set.shape[0])],[test_set[i][-1] for i in range(test_set.shape[0])]

    from sklearn.ensemble import RandomForestClassifier 
    rf = RandomForestClassifier() 
    rf.fit(X_train, y_train) 

    feature_importances.append(rf.feature_importances_)

avg_feature_importances=sum(feature_importances)/len(feature_importances)
avg_feature_importances=pd.DataFrame(avg_feature_importances*100,columns=['Average Importance(%)'],index=list(data.columns[:-1]))

print(avg_feature_importances.sort_values('Average Importance(%)',ascending=False))
cols_to_use=[avg_feature_importances.index[i] for i in range(len(avg_feature_importances.index)) if (avg_feature_importances.iloc[i,0]>=(avg_feature_importances).mean()[0])==True]
cols_to_use.append('Target')

data=pd.read_csv(filepath_or_buffer=path,usecols=cols_to_use)
data=remove_outliers(data)

corr_matrix = data.iloc[:,:-1].corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_remove = [column for column in upper.columns if any(upper[column] > 0.85)]
[cols_to_use.remove(col) for col in to_remove];

                            Average Importance(%)
Exudates_Detection_1                     9.218834
MA_Detection_alpha-0.5                   8.255313
Macula_OpticDisc_Distance                7.816744
OpticDisc_Diameter                       7.622605
Exudates_Detection_2                     7.464093
Exudates_Detection_3                     7.215708
Exudates_Detection_4                     6.785061
MA_Detection_alpha-0.6                   6.335278
MA_Detection_alpha-0.7                   5.788920
MA_Detection_alpha-0.9                   5.489277
MA_Detection_alpha-1.0                   5.439081
MA_Detection_alpha-0.8                   5.344553
Exudates_Detection_5                     4.951660
Exudates_Detection_7                     4.432978
Exudates_Detection_6                     3.856670
Exudates_Detection_8                     3.138794
AM/FM-based classification               0.844432
Pre-Screening                            0.000000
Quality_Assessment                       0.000000


In [5]:
def main():    
    for _ in range(n_rounds):
        data=pd.read_csv(filepath_or_buffer=path,usecols=cols_to_use)
        data=remove_outliers(data)
        
        train_set, test_set = splitDataset(dataset=data,trainRatio=trainRatio)
        X_test=[test_set[i][:-1] for i in range(test_set.shape[0])]
        y_test=[test_set[i][-1] for i in range(test_set.shape[0])]

        preds=make_pred(data,np.array(X_test))      
        
        cm = confusion_matrix(y_test, preds)
        sr = 100*(cm.diagonal().sum()/cm.sum())
        confusion_matrixes.append(cm)
        accuracys.append(sr)
        
        class0_success=100*cm[0][0]/(cm[0][0]+cm[0][1])
        class1_success=100*cm[1][1]/(cm[1][0]+cm[1][1])
         
        class0_successes.append(class0_success)
        class1_successes.append(class1_success)

        if ((_+1)%(int(n_rounds/10))==0):
            print(f'{(_+1)*(100/n_rounds):05.2f}% concluído!')

In [6]:
accuracys = []
confusion_matrixes = []
class0_successes,class1_successes=[],[]
n_rounds=100

%time main()

10.00% concluído!
20.00% concluído!
30.00% concluído!
40.00% concluído!
50.00% concluído!
60.00% concluído!
70.00% concluído!
80.00% concluído!
90.00% concluído!
100.00% concluído!
Wall time: 6min 27s


# Classificador Quadrático
<br>

Para cada classe $C_i \in C=\{C_1,C_2,\dots,C_K\}$ calculamos o vetor centróide $m_i$ como:
$$m_i = \dfrac{1}{N_i}\sum_{x\in C_i}x$$

e atribuímos uma observação a classe $i$ se, $\forall j \neq i$,

$$(x-m_i)^T C^{-1}_i(x-m_i) < (x-m_j)^T C^{-1}_j(x-m_j)$$

onde $C^{-1}_i$ denota a inversa da matriz de covariância da classe i, com $i,j \in [1,K]$.



In [7]:
Accuracys=pd.DataFrame(data={'Accuracys':accuracys})
Accuracys.describe()

,Accuracys
count,100.000000
mean,61.897917
std,1.702553
min,57.500000
25%,60.625000
50%,62.083333
75%,62.916667
max,66.041667


In [8]:
Class_Sucesses=pd.DataFrame(data={'Succces 0':class0_successes,'Succces 1':class1_successes})
Class_Sucesses.describe()

,Succces 0,Succces 1
count,100.000000,100.000000
mean,49.669482,74.189712
std,2.433700,2.019160
min,43.983402,68.951613
25%,48.100822,72.852000
50%,49.687276,74.423368
75%,51.080428,75.240204
max,55.555556,79.295154
